# Run Cross-Sectional Model

## Imports

In [1]:
# general
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# my packaes
import utils.colonflag.feature_generation as fg
import utils.missing_data as md
# data preparation
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# models
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


# evaluation
import sklearn.metrics as metrics
from sklearn.model_selection import GridSearchCV
 #confusion_matrix, ConfusionMatrixDisplay,roc_auc_score, average_precision_score, f1_score, classification_report, precision_score, recall_score, roc_curve
from sklearn.model_selection import cross_validate
#utils
import joblib
from sys import getsizeof

#from scipy.stats import linregress


## Set Variables

In [2]:
dir = r"C:\Users\victo\OneDrive - University of Leeds\Documents\Uni Work\Project\MIMIC Work\Liver Cancer Prediction\liver_pred\data\interim"
model_dir = r"C:\Users\victo\OneDrive - University of Leeds\Documents\Uni Work\Project\MIMIC Work\Liver Cancer Prediction\liver_pred\data\models"

## Load Data

In [3]:
#cohort_ids = pd.read_csv(dir + r"\matched_cohort_ids.csv", index_col=0)
processed_labs = pd.read_csv(dir + r"\processed_lab_data.csv", parse_dates=["charttime", "index_date"], index_col=0,
                             usecols=['subject_id',  'index_admission',  'test_admission', 'itemid', 'valuenum', 'charttime', 'outcome', 'index_date', 'label'],
                             dtype = {'subject_id':int, 'index_admission':int, 'test_admission': pd.Int32Dtype(), 'itemid':int,
                                      'valuenum': 'float64','outcome': bool,'label': 'category'})
# pd.Int32Dtype() is used to allow for NaN values in the test_admission column

Functions for 

In [4]:
def prepare_features(feature_df):

    print("Preparing Features")
    feature_df = feature_df.fillna(0, inplace=False)
    y = feature_df['outcome']
    X = feature_df.drop(columns=["outcome"])
    X_train, X_test,y_train, y_test = train_test_split(X, y, train_size = 0.8)
    print(f"Train Length: {len(X_train)}        Train cases: {len(y_train[y_train==1])}    Proportion: {len(y_train[y_train==1])/len(y_train)*100} %")
    print(f"Test Length: {len(X_test)}          Test cases: {len(y_test[y_test==1])}       Proportion: {len(y_test[y_test==1])/len(y_test)*100} %")

    #### Scale training #####
    scaler = StandardScaler()
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train))
    X_train_scaled = X_train_scaled.set_axis(X_train.columns,axis=1)

    #### Scale Test using training scaler ####
    X_test_scaled = pd.DataFrame(scaler.transform(X_test))
    X_test_scaled.set_axis(X_test.columns,axis=1)

    return X_train_scaled, X_test_scaled, y_train, y_test


In [5]:
def evaluate_performance(y_probs, y_true):
    """
    Evaluate the performance of a binary classification model by calculating various metrics.

    Parameters:
    - y_probs (array-like): Predicted probabilities for the positive class.
    - y_true (array-like): True labels for the samples.

    Returns:
    - f1 (float): F1 score.
    - roc_auc (float): ROC AUC score.
    - precision (float): Precision score.
    - recall (float): Recall score.
    """
    y_pred = y_probs > 0.5
    f1 = metrics.f1_score(y_true, y_pred)
    print(f"F1 Score: {f1}")
    roc_auc = metrics.roc_auc_score(y_true, y_probs)
    print(f"ROC AUC: {roc_auc}")
    precision = metrics.precision_score(y_true, y_pred)
    print(f"Precision: {precision}")
    recall = metrics.recall_score(y_true, y_pred)
    print(f"Recall: {recall}")
    confusion_matrix = metrics.confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:", confusion_matrix)
    return f1, roc_auc, precision, recall

In [6]:
def tune_hyperparameters(model, param_grid, X_train, y_train):
    print(f"Running {model}")
    clf = GridSearchCV(model, param_grid, cv=5, n_jobs=-1)
    clf.fit(X_train, y_train)
    joblib.dump(clf, model_dir+r"\bestparams_" + f"{model}.pkl")
    y_pred = clf.predict(X_train)
    y_prob = clf.predict_proba(X_train)[:,1]
    return y_pred, y_prob, clf

## Run models on *training set only* with no lead time to find optimal parameters

In [7]:
models = {'LR':LogisticRegression(), 'RF':RandomForestClassifier(), 'GB': GradientBoostingClassifier(), 'NN': MLPClassifier(), 'SVM':SVC(probability=True)}

In [ ]:
lab_df_3day = fg.current_bloods_df(processed_labs,lead_time=0, n_days_pre=3, n_days_post=1)
X_train, _, y_train, _ = prepare_features(lab_df_3day)

Preparing Features
Train Length: 6251        Train cases: 1087    Proportion: 17.389217725163974 %
Test Length: 1563          Test cases: 271       Proportion: 17.338451695457454 %


c:\Users\victo\OneDrive - University of Leeds\Documents\Uni Work\Project\MIMIC Work\Liver Cancer Prediction\liver_pred\liver_pred\utils\colonflag\feature_generation.py:126: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  current = pd.concat([current, missing_data])


### Tune LR

In [ ]:
# Define parameter grid for logistic regression
param_grid_lr = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}

In [ ]:
y_pred, y_prob, lr = tune_hyperparameters(models['LR'], param_grid_lr, X_train, y_train)
results = evaluate_performance(y_prob, y_train)

Running LogisticRegression()
F1 Score: 0.5380184331797235
ROC AUC: 0.8747077460046448
Precision: 0.7195685670261941
Recall: 0.4296228150873965
Confusion Matrix: [[4982  182]
 [ 620  467]]


### Tune RF

In [ ]:
# Define parameter grid for random forest
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5, 10], 
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']}


In [4]:
y_pred, y_prob, RF = tune_hyperparameters(models['RF'], param_grid_rf, X_train, y_train)
results = evaluate_performance(y_prob, y_train)

NameError: name 'tune_hyperparameters' is not defined

### Tune GB

In [5]:
# Define parameter grid for gradient boosting
param_grid_gb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 1],
    'max_depth': [3, 5, 7, 9],
    'subsample': [0.5, 0.7, 1.0],
    'max_features': ['auto', 'sqrt', 'log2']
}

In [6]:
y_pred, y_prob, GB = tune_hyperparameters(models['GB'], param_grid_gb, X_train, y_train)
results = evaluate_performance(y_prob, y_train)

NameError: name 'tune_hyperparameters' is not defined

### Tune NN

In [7]:
# Define parameter grid for neural network
param_grid_nn = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}


In [ ]:
y_pred, y_prob, NN = tune_hyperparameters(models['NN'], param_grid_nn, X_train, y_train)
results = evaluate_performance(y_prob, y_train)

### Tune SVM

In [ ]:
param_grid_svc = {
    'C': [0.1, 1, 10, 100, 1000],
    'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
    'kernel': ['rbf', 'poly']
}

NOTE: SVM doesn't estimate probabilities, that's done using validation. Adjust the evaluation functions?

In [ ]:
y_pred, y_prob, SVM = tune_hyperparameters(models['SVM'], param_grid_svc, X_train, y_train)
results = evaluate_performance(y_prob, y_train)

Best models

In [13]:
lr_best = joblib.load(model_dir+r"\bestparams_LogisticRegression().pkl")
#rf_best = joblib.load(model_dir+r"\bestparams_RandomForestClassifier().pkl")
#gb_best = joblib.load(model_dir+r"\bestparams_GradientBoostingClassifier().pkl")
#nn_best = joblib.load(model_dir+r"\bestparams_MLPClassifier().pkl")
#svm_best = joblib.load(model_dir+r"\bestparams_SVC(probability=True).pkl")


## Early detection

Evaluate model with different lead times.

In [ ]:
def evaluate_lead_times(lead_times, processed_labs):

    lr_aucs = np.zeros(len(lead_times))
    lr_precisions = np.zeros(len(lead_times))
    lr_recalls = np.zeros(len(lead_times))
    n_iter = 0
    iter_times = np.zeros(len(lead_times))
    for l_time in lead_times:
        lab_df = fg.current_bloods_df(processed_labs, lead_time = l_time, n_days_pre=3, n_days_post=1)
        X_train, _, y_train, _ = prepare_features(lab_df)
        lab_df = None
        results = cross_validate(lr_best, X_train, y_train, scoring = ('roc_auc', 'precision', 'recall'), cv=5, n_jobs=2)
        X_train = y_train = None
        lr_aucs[n_iter] = results['test_roc_auc'].mean()
        lr_precisions[n_iter] = results['test_precision'].mean()
        lr_recalls[n_iter] = results['test_recall'].mean()
        iter_time = sum(results['fit_time'])+sum(results['score_time'])
        iter_times[n_iter] = iter_time
        print("Iter:", n_iter+1, "Iter time:", iter_time, "Estimated Total:", sum(iter_times)/len(iter_times)*len(lead_times)/60, "mins")
        n_iter +=1
    return lr_aucs, lr_precisions, lr_recalls, iter_times

In [14]:
lead_times = range(0,735,30)

In [ ]:
lr_aucs, lr_precisions, lr_recalls, iter_times = evaluate_lead_times(lead_times, processed_labs)

Plot LR results

In [ ]:
# Plot AUCs
plt.plot(lead_times, lr_aucs, label='Logistic Regression')
plt.xlabel('Lead Time (days)')
plt.ylabel('AUC')
plt.legend()
plt.show()

# Plot precisions
plt.plot(lead_times, lr_precisions, label='Logistic Regression')
plt.xlabel('Lead Time (days)')
plt.ylabel('Precision')
plt.legend()
plt.show()

# Plot recalls
plt.plot(lead_times, lr_recalls, label='Logistic Regression')
plt.xlabel('Lead Time (days)')
plt.ylabel('Recall')
plt.legend()
plt.show()

In [ ]:
import timeit
import pandas as pd